In [229]:
!pip install webdriver_manager

In [230]:
# imports
import requests as rq
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from selenium import webdriver
import os
from webdriver_manager.chrome import ChromeDriverManager
import unicodedata
# end imports

# Part 01  : Data Gathering

In [231]:
# Settings 
chrome_driver_path = '/Users/chihab_eddine98/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver'

reviews_div_cls = 'review'

prod_url = 'https://www.amazon.fr/Microsoft-Surface-Pro-Hybride-stockage/dp/B07X8NL7MW/ref=sr_1_7?__mk_fr_FR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&dchild=1&keywords=Microsoft+Surface&qid=1616930888&s=computers&sr=1-7'

prod_urls = {'informatique':[
    {
        'prod_url':
        'https://www.amazon.fr/Microsoft-Surface-Pro-Hybride-stockage/dp/B07X8NL7MW/ref=sr_1_7? __mk_fr_FR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&dchild=1&keywords=Microsoft+Surface&qid=1616930888&  s=computers&sr=1-7',
    'rev_url': 'https://www.amazon.fr/Microsoft-Surface-Pro-Hybride-stockage/product-reviews/B07X8NL7MW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'
    
    }
    ]
}

ds_cols = ['Rev_Title','Rev_Rate','Rev_Bdy','Rev_Hlp','Prod_Title','Prod_Brnd','Prod_Rate','Prod_Eval','Prod_Price','Category']


# End Settings

# Scrapping Product Information

In [232]:

# Scrapping Product Data 
def product_scrapper(driver,prod_category,prod_url):
    driver.get(prod_url)
    prod_soup = BeautifulSoup(driver.page_source,'html.parser')

    # Product Info
    prod_title = prod_soup.find('span',{'id' : 'productTitle' }).text
    prod_brand = prod_soup.find('a',{'id' : 'bylineInfo' }).text
    prod_rate = prod_soup.find('span',{'class' : 'a-icon-alt' }).text
    prod_eval = prod_soup.find('span',{'id' : 'acrCustomerReviewText' }).text
    prod_price = prod_soup.find('span',{'id' : 'priceblock_ourprice' }).text


    result = tuple(map(lambda x : unicodedata.normalize("NFKD",x.strip()),[prod_title,prod_brand,       prod_rate,prod_eval,prod_price,prod_category]))
    return result



## Test Product Scrapping

In [233]:
driver = webdriver.Chrome(chrome_driver_path)

for category,products in prod_urls.items():
    for prod in products:
        product = product_scrapper(driver,category,prod['prod_url'])
        print(product)

('Microsoft Surface Pro 7 PC portable (Windows 10, Écran Tactile 12.3", Intel Core i5, 8Go RAM, 128Go SSD, Platine)', 'Marque : Microsoft Surface', '4,6 sur 5 étoiles', '3 105 évaluations', '855,20 €', 'informatique')


# Scrapping Reviews Data

In [234]:
# Scrapping Reviews Data
def reviews_scrapper(driver,prod,rev_url,ds):
    driver.get(rev_url)
    rev_soup = BeautifulSoup(driver.page_source,'html.parser')
    reviews_div = rev_soup.find_all('div',{'class':reviews_div_cls})

    for review in reviews_div:
        review_title = review.find('a',{'data-hook' : 'review-title'}).text
        review_rate = review.find('i',{'data-hook' : 'review-star-rating'}).text
        review_body = review.find('span',{'data-hook' : 'review-body'}).text
        review_help = review.find('span',{'data-hook' : 'helpful-vote-statement'}).text
        rev_result = tuple(map(lambda x : unicodedata.normalize("NFKD",x.strip()),[review_title,review_rate,review_body,review_help]))
        ds.append(rev_result+prod)

    return ds

## Test Reviews Scrapping

In [235]:
ds = []
for category,products in prod_urls.items():
    for prod in products:
        product = product_scrapper(driver,category,prod['prod_url'])
        ds = reviews_scrapper(driver,product,prod['rev_url'],ds) 

df = pd.DataFrame(ds)
df.columns = ds_cols
df





,Rev_Title,Rev_Rate,Rev_Bdy,Rev_Hlp,Prod_Title,Prod_Brnd,Prod_Rate,Prod_Eval,Prod_Price,Category
0,Surface Pro 7ème génération,"4,0 sur 5 étoiles",Je possède la Surface Pro 6 (soit la généra...,155 personnes ont trouvé cela utile,Microsoft Surface Pro 7 PC portable (Windows 1...,Marque : Microsoft Surface,"4,6 sur 5 étoiles",3 105 évaluations,"855,20 €",informatique
1,Du reconditionné au prix du neuf!,"1,0 sur 5 étoiles",Produit bien reçu.Par contre on achète au pr...,107 personnes ont trouvé cela utile,Microsoft Surface Pro 7 PC portable (Windows 1...,Marque : Microsoft Surface,"4,6 sur 5 étoiles",3 105 évaluations,"855,20 €",informatique
2,THROTTING,"2,0 sur 5 étoiles",POUR LA VERSION I5 SEULEMENT : PHENOMENE DE TH...,71 personnes ont trouvé cela utile,Microsoft Surface Pro 7 PC portable (Windows 1...,Marque : Microsoft Surface,"4,6 sur 5 étoiles",3 105 évaluations,"855,20 €",informatique
3,ATTENTION AU TITRE TROMPEUR !,"2,0 sur 5 étoiles",Le titre fait référence à un Surface Pro 7 ...,98 personnes ont trouvé cela utile,Microsoft Surface Pro 7 PC portable (Windows 1...,Marque : Microsoft Surface,"4,6 sur 5 étoiles",3 105 évaluations,"855,20 €",informatique
4,C'est bien une version i7,"5,0 sur 5 étoiles",Contrairement au commentaire disant que c'est ...,44 personnes ont trouvé cela utile,Microsoft Surface Pro 7 PC portable (Windows 1...,Marque : Microsoft Surface,"4,6 sur 5 étoiles",3 105 évaluations,"855,20 €",informatique
5,Un excellent choix,"5,0 sur 5 étoiles","Pour faire court, sans aucun doute mon ""pc por...",23 personnes ont trouvé cela utile,Microsoft Surface Pro 7 PC portable (Windows 1...,Marque : Microsoft Surface,"4,6 sur 5 étoiles",3 105 évaluations,"855,20 €",informatique
6,Plus pertinent qu'une tablette sous Android.,"5,0 sur 5 étoiles",Belle bête ! J'essaie pour la 1ere fois Windo...,22 personnes ont trouvé cela utile,Microsoft Surface Pro 7 PC portable (Windows 1...,Marque : Microsoft Surface,"4,6 sur 5 étoiles",3 105 évaluations,"855,20 €",informatique
7,Eclan bleu systématique au démarrage ou dans...,"1,0 sur 5 étoiles","Sans qu'aucun logiciel n'ai été installé, j...",10 personnes ont trouvé cela utile,Microsoft Surface Pro 7 PC portable (Windows 1...,Marque : Microsoft Surface,"4,6 sur 5 étoiles",3 105 évaluations,"855,20 €",informatique
8,Incroyable,"4,0 sur 5 étoiles",Cette tablette est incroyable et presque parfa...,Une personne a trouvé cela utile,Microsoft Surface Pro 7 PC portable (Windows 1...,Marque : Microsoft Surface,"4,6 sur 5 étoiles",3 105 évaluations,"855,20 €",informatique
9,Tablette fonctionne bien,"1,0 sur 5 étoiles",UPDATE: La durée de vie de la batterie est de...,4 personnes ont trouvé cela utile,Microsoft Surface Pro 7 PC portable (Windows 1...,Marque : Microsoft Surface,"4,6 sur 5 étoiles",3 105 évaluations,"855,20 €",informatique
